In [5]:
import pandas as pd
import glob
import os
import re
from typing import Dict, List, Tuple

parent_dir = os.path.dirname(os.getcwd())
# date = "10.28"
date_in_filename = os.path.basename(parent_dir)
date=date_in_filename.split("_")[0]

save_dir = os.path.join(parent_dir, "tex")
if not os.path.exists(save_dir):
    os.makedirs(save_dir, exist_ok=True)

data_dir = os.path.join(parent_dir, "processed_data")
raw_export_dir = os.path.join(parent_dir, "raw_export")
raw_export_files=[]

# Get session id list from data directory
session_id_list = []

for root, dirs, files in os.walk(data_dir):
    for file in files:
        file_path = os.path.join(root, file)
        session_id=file_path.split('clean_')[-1].rstrip('.csv')
        print(session_id)
        session_id_list.append(session_id)


print(session_id_list)

for root, dirs, files in os.walk(raw_export_dir):
    for file in files:
        file_path = os.path.join(root, file)
        if file_path.split('raw_export/')[-1].split('_')[0] == 'raw':
            raw_export_files.append(file_path)

print(raw_export_files)

phi4_85a9_psg_batch
phi4_85a9_tpusg_batch
['phi4_85a9_psg_batch', 'phi4_85a9_tpusg_batch']
['/Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/07.30/raw_export/raw_phi4_85a9_tpusg_batch.json', '/Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/07.30/raw_export/raw_phi4_85a9_psg_batch.json']


In [6]:
import json

def extract_successful_sketches(filepath):
    """
    Extract output.sketch from data entries where output.status is 'success'
    """
    with open(filepath, 'r') as f:
        data = json.load(f)
    
    successful_sketches = []
    
    for entry_idx, entry in enumerate(data.get('data', [])):
        entry_id = entry.get('id', f'entry_{entry_idx}')
        entry_output = entry.get('output', {})
        
        if entry_output and entry_output.get('status') == 'success':
            sketch = entry_output.get('sketch')
            if sketch:
                sketch_info = {
                    'entry_id': entry_id,
                    'entry_index': entry_idx,
                    'session_id': entry.get('session_id', 'unknown'),
                    'name': entry.get('name', 'unnamed'),
                    'timestamp': entry.get('timestamp', 'unknown'),
                    'model': entry.get('model', 'unknown'),
                    'sketch': sketch,
                    'sketch_length': len(sketch) if isinstance(sketch, str) else 'not_string',
                    'tags': entry.get('tags', [])
                }
                successful_sketches.append(sketch_info)
    
    return successful_sketches

def analyze_sketch_structure(sketch_data):
    """
    Analyze the structure of sketch data to understand what it contains
    """
    if isinstance(sketch_data, str):
        print(f"Sketch is a string with {len(sketch_data)} characters")
        # print(f"Preview: {sketch_data[:300]}...")
        
        # Check if it looks like code
        if 'import' in sketch_data or 'def ' in sketch_data or 'class ' in sketch_data:
            # print("✓ Appears to contain code")
            pass
        
        # Check for common file extensions or formats
        if sketch_data.startswith('{') and sketch_data.endswith('}'):
            print("✓ Appears to be JSON format")
        elif '```' in sketch_data:
            print("✓ Appears to contain markdown code blocks")
            
    elif isinstance(sketch_data, dict):
        print(f"Sketch is a dictionary with keys: {list(sketch_data.keys())}")
        for key, value in sketch_data.items():
            print(f"  {key}: {type(value)} - {len(str(value))} chars")
    elif isinstance(sketch_data, list):
        print(f"Sketch is a list with {len(sketch_data)} items")
        if sketch_data:
            print(f"First item type: {type(sketch_data[0])}")
    else:
        print(f"Sketch is of type: {type(sketch_data)}")

# Test with the first file
print("=== Extracting Successful Sketches ===")
first_file = raw_export_files[0]
print(f"Analyzing file: {first_file.split('/')[-1]}")

successful_sketches = extract_successful_sketches(first_file)
print(f"Found {len(successful_sketches)} successful sketches")

# Analyze each successful sketch
for i, sketch_info in enumerate(successful_sketches):
    print(f"\n--- Sketch {i+1} ---")
    print(f"Entry ID: {sketch_info['entry_id']}")
    print(f"Session ID: {sketch_info['session_id']}")
    print(f"Name: {sketch_info['name']}")
    # print(f"Tags: {sketch_info['tags']}")
    
    analyze_sketch_structure(sketch_info['sketch'])

=== Extracting Successful Sketches ===
Analyzing file: raw_phi4_85a9_tpusg_batch.json
Found 30 successful sketches

--- Sketch 1 ---
Entry ID: 7813eda3
Session ID: phi4_85a9_tpusg_batch
Name: 7813_tpu_sketch_generator
Sketch is a string with 2862 characters

--- Sketch 2 ---
Entry ID: 4b0689be
Session ID: phi4_85a9_tpusg_batch
Name: 4b06_tpu_sketch_generator
Sketch is a string with 3383 characters

--- Sketch 3 ---
Entry ID: cd1e42b6
Session ID: phi4_85a9_tpusg_batch
Name: cd1e_tpu_sketch_generator
Sketch is a string with 3461 characters

--- Sketch 4 ---
Entry ID: cf6ff531
Session ID: phi4_85a9_tpusg_batch
Name: cf6f_tpu_sketch_generator
Sketch is a string with 3173 characters

--- Sketch 5 ---
Entry ID: a7d40f6a
Session ID: phi4_85a9_tpusg_batch
Name: a7d4_tpu_sketch_generator
Sketch is a string with 3874 characters

--- Sketch 6 ---
Entry ID: d92bae51
Session ID: phi4_85a9_tpusg_batch
Name: d92b_tpu_sketch_generator
Sketch is a string with 3481 characters

--- Sketch 7 ---
Entry ID:

In [7]:
# Let's get a summary across all files first
def analyze_all_files_summary():
    """
    Get a summary of successful sketches across all JSON files
    """
    total_entries = 0
    total_successful = 0
    file_summary = []
    
    for file_path in raw_export_files:
        filename = file_path.split('/')[-1]
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
            
            entries_in_file = len(data.get('data', []))
            successful_in_file = 0
            
            for entry in data.get('data', []):
                if entry.get('output', {}).get('status') == 'success':
                    successful_in_file += 1
            
            total_entries += entries_in_file
            total_successful += successful_in_file
            
            file_summary.append({
                'filename': filename,
                'total_entries': entries_in_file,
                'successful_entries': successful_in_file,
                'success_rate': successful_in_file / entries_in_file if entries_in_file > 0 else 0
            })
            
        except Exception as e:
            print(f"Error processing {filename}: {e}")
    
    return file_summary, total_entries, total_successful

print("=== Summary Across All Files ===")
file_summary, total_entries, total_successful = analyze_all_files_summary()

print(f"Total files: {len(raw_export_files)}")
print(f"Total entries across all files: {total_entries}")
print(f"Total successful entries: {total_successful}")
print(f"Overall success rate: {total_successful/total_entries:.2%}" if total_entries > 0 else "No entries found")

print("\n=== Per-File Summary ===")
for summary in file_summary:
    print(f"{summary['filename']}: {summary['successful_entries']}/{summary['total_entries']} ({summary['success_rate']:.1%})")

# Now let's look at a few specific successful sketches
print("\n=== Sample Successful Sketches ===")
sample_count = 0
max_samples = 3

for file_path in raw_export_files:
    if sample_count >= max_samples:
        break
        
    sketches = extract_successful_sketches(file_path)
    if sketches:
        filename = file_path.split('/')[-1]
        print(f"\nFrom {filename}:")
        
        for sketch_info in sketches[:2]:  # Show max 2 per file
            if sample_count >= max_samples:
                break
                
            print(f"  Entry: {sketch_info['name']}")
            print(f"  Session: {sketch_info['session_id']}")
            
            sketch = sketch_info['sketch']
            if isinstance(sketch, str):
                # number of lines
                total_lines = len(sketch.split('\n'))
                print(f"Sketch length: {total_lines} lines")
                # print(f"  Sketch length: {len(sketch)} chars")
                # Show just first few lines
                # lines = sketch.split('\n')[:5]
                # for line in lines:
                #     print(f"    {line[:100]}")
                # total_lines = len(sketch.split('\n'))
                # if total_lines > 5:
                #     remaining_lines = total_lines - 5
                #     print(f"    ... ({remaining_lines} more lines)")
            else:
                print(f"  Sketch type: {type(sketch)}")
                # print(f"  Sketch content: {str(sketch)[:200]}...")
            
            sample_count += 1
            print()

=== Summary Across All Files ===
Total files: 2
Total entries across all files: 60
Total successful entries: 48
Overall success rate: 80.00%

=== Per-File Summary ===
raw_phi4_85a9_tpusg_batch.json: 30/30 (100.0%)
raw_phi4_85a9_psg_batch.json: 18/30 (60.0%)

=== Sample Successful Sketches ===

From raw_phi4_85a9_tpusg_batch.json:
  Entry: 7813_tpu_sketch_generator
  Session: phi4_85a9_tpusg_batch
Sketch length: 80 lines

  Entry: 4b06_tpu_sketch_generator
  Session: phi4_85a9_tpusg_batch
Sketch length: 91 lines


From raw_phi4_85a9_psg_batch.json:
  Entry: c802_py_sketch_generator
  Session: phi4_85a9_psg_batch
Sketch length: 94 lines



In [8]:
mandatory_variables = {
    'tpusg':[
        '/home/mendel/tinyml_autopilot/models/edgetpu_detect.tflite',
        '/home/mendel/tinyml_autopilot/models/labelmap.txt',
        '/home/mendel/tinyml_autopilot/data//sheeps.mp4',
        '/home/mendel/tinyml_autopilot/results/sheeps_detections.mp4'],
    'psg':['models/ssd-mobilenet_v1/detect.tflite',
           'models/ssd-mobilenet_v1/labelmap.txt',
           'data/object_detection/sheeps.mp4',
           'results/object_detection/test_results/sheeps_detections.mp4']}

def save_successful_sketches_to_files():
    """
    Extract all successful sketches and save them as individual Python files
    """
    # Create output directory
    output_dir = os.path.join(parent_dir, "extracted_code")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
    
    saved_files = []
    sketch_counter = 0
    
    for file_path in raw_export_files:
        filename = file_path.split('/')[-1].replace('.json', '')
        sketches = extract_successful_sketches(file_path)
        
        for sketch_info in sketches:
            sketch_counter += 1
            
            # Create descriptive filename
            entry_name = sketch_info['name']
            session_id = sketch_info['session_id']
            entry_id = sketch_info['entry_id']
            
            # Clean up the entry name for filename
            clean_name = re.sub(r'[^a-zA-Z0-9_]', '_', entry_name)
            if "tpu_sketch_generator" in clean_name:
                clean_name = clean_name.replace("tpu_sketch_generator", "tpusg").split('_')[1]
            elif "py_sketch_generator" in clean_name:
                clean_name = clean_name.replace("py_sketch_generator", "psg").split('_')[1]
            # qwen2.5-coder:32b_ae24_tpsg_batch

            model_name= session_id.split('_')[0]
            if model_name == 'qwen2.5-coder:32b':
                model_name = 'qwen32'
            elif model_name == 'qwen2.5-coder:14b':
                model_name = 'qwen14'
            elif ':latest' in model_name:
                model_name = model_name.split(':')[0]
                
            model_name=model_name.replace('_','-')
            session_uid = session_id.split('_')[1]
            
            # Create filename
            py_filename = f"{sketch_counter:03d}_{session_uid}_{model_name}_{clean_name}_{entry_id}.py"
            py_filepath = os.path.join(output_dir, py_filename)
            
            # Add metadata header to the file
            header = f'''"""
Extracted from: {filename}
Entry ID: {entry_id}
Entry Name: {entry_name}
Session ID: {session_id}
Timestamp: {sketch_info['timestamp']}
Tags: {', '.join(sketch_info['tags'])}
"""

''' 
            if len(sketch_info['sketch'].splitlines()) < 50:
                print(ValueError(f"❌ Sketch {sketch_counter:03d} is too short: {entry_id[:4]}.\n "))
                continue
            
            
            # Check if mandatory variables are present, the two keys are mapped into the clean_name, depending on which being presented. also tell which is missing
            missing_vars = [var for var in mandatory_variables[clean_name] if var not in sketch_info['sketch']]
            if missing_vars:
                print(ValueError(f"❌ Mandatory variables missing in sketch {sketch_counter:03d}: {entry_id[:4]}.\n Missing: {', '.join(missing_vars)}"))
                continue
            if 'results/sheeps_detections.mp4' not in sketch_info['sketch']:
                # print the first 20 lines of sketch_info['sketch']
                # first_20_lines = '\n'.join(sketch_info['sketch'].split('\n')[:20])
                print(ValueError(f"❌ Output path not found in sketch {sketch_counter:03d}: {entry_id[:4]}.\n ")) 
                continue
            else:
                sketch_info['sketch'] = sketch_info['sketch'].replace('results/sheeps_detections.mp4', f'results/sheeps_detections_{entry_id[:4]}.mp4')
                
                
            # Save the sketch
            with open(py_filepath, 'w', encoding='utf-8') as f:
                f.write(header)
                f.write(sketch_info['sketch'])
            
            saved_files.append({
                'filepath': py_filepath,
                'filename': py_filename,
                'entry_id': entry_id,
                'entry_name': entry_name,
                'session_id': session_id,
                'source_file': filename,
                'sketch_length': len(sketch_info['sketch'])
            })
    
    return saved_files, output_dir

# Save all successful sketches
print("=== Saving Successful Sketches to Python Files ===")
saved_files, output_dir = save_successful_sketches_to_files()

print(f"Saved {len(saved_files)} Python files to: {output_dir}")
# print(f"Total characters extracted: {sum(f['sketch_length'] for f in saved_files):,}")

print("\n=== Saved Files Summary ===")
for i, file_info in enumerate(saved_files[:10]):  # Show first 10
    print(f"{i+1:2d}. {file_info['filename']}")
    print(f"     Source: {file_info['source_file']}")
    print(f"     Entry: {file_info['entry_name']}")
    # print(f"     Length: {file_info['sketch_length']:,} chars")

if len(saved_files) > 10:
    print(f"     ... and {len(saved_files) - 10} more files")

print(f"\nAll files saved in: {output_dir}")

=== Saving Successful Sketches to Python Files ===
❌ Mandatory variables missing in sketch 031: c802.
 Missing: models/ssd-mobilenet_v1/labelmap.txt
❌ Mandatory variables missing in sketch 032: 8c80.
 Missing: data/object_detection/sheeps.mp4, results/object_detection/test_results/sheeps_detections.mp4
❌ Sketch 034 is too short: 7a05.
 
❌ Mandatory variables missing in sketch 037: a0cf.
 Missing: models/ssd-mobilenet_v1/detect.tflite, models/ssd-mobilenet_v1/labelmap.txt, data/object_detection/sheeps.mp4, results/object_detection/test_results/sheeps_detections.mp4
❌ Sketch 039 is too short: e043.
 
❌ Sketch 047 is too short: 5956.
 
Saved 42 Python files to: /Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/07.30/extracted_code

=== Saved Files Summary ===
 1. 001_85a9_phi4_tpusg_7813eda3.py
     Source: raw_phi4_85a9_tpusg_batch
     Entry: 7813_tpu_sketch_generator
 2. 002_85a9_phi4_tpusg_4b0689be.py
     Source: raw_phi4_85a9_tpusg_batch
     Entry: 4b06_tpu_ske

In [9]:
# Create a comprehensive summary report including quality filtering
def create_extraction_summary_with_quality():
    """
    Create a comprehensive summary of the extraction process including quality filtering
    """
    summary_data = []
    total_successful_raw = 0
    total_qualified = 0
    total_too_short = 0
    total_missing_vars = 0
    total_missing_output = 0
    
    for file_path in raw_export_files:
        filename = file_path.split('/')[-1]
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        total_entries = len(data.get('data', []))
        successful_entries = []
        failed_entries = []
        qualified_entries = []
        too_short_entries = []
        missing_vars_entries = []
        missing_output_entries = []
        
        for entry in data.get('data', []):
            entry_output = entry.get('output', {})
            status = entry_output.get('status', 'unknown')
            
            if status == 'success':
                sketch = entry_output.get('sketch', '')
                entry_name = entry.get('name', 'unnamed')
                
                # Determine clean_name (matching the logic from save function)
                clean_name = re.sub(r'[^a-zA-Z0-9_]', '_', entry_name)
                if "tpu_sketch_generator" in clean_name:
                    clean_name = "tpusg"
                elif "py_sketch_generator" in clean_name:
                    clean_name = "psg"
                else:
                    # Extract the key part if pattern doesn't match
                    clean_name = clean_name.split('_')[1] if '_' in clean_name else clean_name
                
                entry_info = {
                    'id': entry.get('id'),
                    'name': entry_name,
                    'session_id': entry.get('session_id'),
                    'sketch_length': len(sketch),
                    'sketch_lines': len(sketch.splitlines()),
                    'clean_name': clean_name
                }
                successful_entries.append(entry_info)
                total_successful_raw += 1
                
                # Apply quality filters - match the exact logic from save_successful_sketches_to_files()
                is_too_short = len(sketch.splitlines()) < 50
                
                # Check mandatory variables (only if clean_name is in mandatory_variables)
                missing_vars = []
                if clean_name in mandatory_variables:
                    missing_vars = [var for var in mandatory_variables[clean_name] if var not in sketch]
                
                is_missing_output = 'results/sheeps_detections.mp4' not in sketch
                
                if is_too_short:
                    too_short_entries.append(entry_info)
                    total_too_short += 1
                elif missing_vars:  # Only check this if not too short
                    entry_info['missing_vars'] = missing_vars
                    missing_vars_entries.append(entry_info)
                    total_missing_vars += 1
                elif is_missing_output:  # Only check this if previous filters passed
                    missing_output_entries.append(entry_info)
                    total_missing_output += 1
                else:
                    qualified_entries.append(entry_info)
                    total_qualified += 1
            else:
                failed_entries.append({
                    'id': entry.get('id'),
                    'name': entry.get('name'),
                    'status': status,
                    'error': entry_output.get('last_error', 'No error info')[:100] + '...' if entry_output.get('last_error') else 'No error info'
                })
        
        summary_data.append({
            'filename': filename,
            'total_entries': total_entries,
            'successful_count': len(successful_entries),
            'failed_count': len(failed_entries),
            'qualified_count': len(qualified_entries),
            'too_short_count': len(too_short_entries),
            'missing_vars_count': len(missing_vars_entries),
            'missing_output_count': len(missing_output_entries),
            'success_rate': len(successful_entries) / total_entries if total_entries > 0 else 0,
            'qualification_rate': len(qualified_entries) / len(successful_entries) if successful_entries else 0,
            'successful_entries': successful_entries,
            'failed_entries': failed_entries,
            'qualified_entries': qualified_entries,
            'too_short_entries': too_short_entries,
            'missing_vars_entries': missing_vars_entries,
            'missing_output_entries': missing_output_entries
        })
    
    return summary_data, total_successful_raw, total_qualified, total_too_short, total_missing_vars, total_missing_output

# Generate and display enhanced summary
print("=== ENHANCED EXTRACTION SUMMARY REPORT ===\n")
summary, total_successful_raw, total_qualified, total_too_short, total_missing_vars, total_missing_output = create_extraction_summary_with_quality()

total_across_all = sum(s['total_entries'] for s in summary)

print(f"📊 OVERALL STATISTICS")
print(f"   Total JSON files processed: {len(summary)}")
print(f"   Total entries across all files: {total_across_all}")
print(f"   Total successful extractions: {total_successful_raw}")
print(f"   Overall success rate: {total_successful_raw/total_across_all:.1%}")
print(f"   ")
print(f"🔍 QUALITY FILTERING RESULTS")
print(f"   Successful sketches (raw): {total_successful_raw}")
print(f"   ✅ Qualified sketches (saved): {total_qualified}")
print(f"   ❌ Too short (< 50 lines): {total_too_short}")
print(f"   ❌ Missing mandatory variables: {total_missing_vars}")
print(f"   ❌ Missing output path: {total_missing_output}")
print(f"   Quality pass rate: {total_qualified/total_successful_raw:.1%}" if total_successful_raw > 0 else "N/A")

print(f"\n📁 FILES PROCESSED (Success → Qualified)")
for s in summary:
    success_count = s['successful_count']
    qualified_count = s['qualified_count']
    too_short = s['too_short_count']
    missing_vars = s['missing_vars_count']
    missing_output = s['missing_output_count']
    
    print(f"   {s['filename']}:")
    print(f"     Success: {success_count}/{s['total_entries']} ({s['success_rate']:.1%})")
    print(f"     Qualified: {qualified_count}/{success_count} ({s['qualification_rate']:.1%})" if success_count > 0 else "     Qualified: 0/0 (N/A)")
    if too_short > 0:
        print(f"     ❌ Too short: {too_short}")
        # Show details for too short entries
        for entry in s['too_short_entries']:
            print(f"        - {entry['id'][:4]}: {entry['sketch_lines']} lines")
    if missing_vars > 0:
        print(f"     ❌ Missing mandatory vars: {missing_vars}")
        # Show details for missing variables entries
        for entry in s['missing_vars_entries']:
            missing_list = ', '.join(entry.get('missing_vars', []))
            print(f"        - {entry['id'][:4]} ({entry['clean_name']}): {missing_list}")
    if missing_output > 0:
        print(f"     ❌ Missing output: {missing_output}")

print(f"\n💾 OUTPUT")
print(f"   Extracted sketches saved to: {output_dir}")
print(f"   Number of Python files created: {len(saved_files)}")

print(f"\n🔍 WHAT WAS EXTRACTED")
print("   Each qualified sketch contains:")
print("   - Complete Python code for TinyML applications (≥50 lines)")
print("   - All mandatory variables for the application type")
print("   - Required output path: 'results/sheeps_detections.mp4'")
print("   - Metadata header with source information")
print("   - Entry ID, session ID, and timestamps")
print("   - Original entry name and tags")

# Show session breakdown for qualified sketches
session_counts = {}
session_qualified = {}
session_total_successful = {}

for s in summary:
    for entry in s['successful_entries']:
        session = entry['session_id']
        session_total_successful[session] = session_total_successful.get(session, 0) + 1
    
    for entry in s['qualified_entries']:
        session = entry['session_id']
        session_qualified[session] = session_qualified.get(session, 0) + 1

for file_info in saved_files:
    session = file_info['session_id']
    session_counts[session] = session_counts.get(session, 0) + 1

print(f"\n🏷️  SESSIONS QUALITY BREAKDOWN")
for session in session_total_successful.keys():
    total_success = session_total_successful.get(session, 0)
    qualified = session_qualified.get(session, 0)
    qualification_rate = qualified / total_success if total_success > 0 else 0
    print(f"   {session}:")
    print(f"     Successful: {total_success}, Qualified: {qualified} ({qualification_rate:.1%})")

print(f"\n✅ Extraction complete! Check the '{output_dir}' directory for all qualified Python files.")
print(f"📋 Quality criteria applied (in order):")
print(f"   1. Minimum 50 lines of code")
print(f"   2. Must contain all mandatory variables for application type:")
for app_type, vars_list in mandatory_variables.items():
    print(f"      • {app_type}: {len(vars_list)} variables required")
print(f"   3. Must contain 'results/sheeps_detections.mp4' output path")
print(f"   • Note: Filters are applied sequentially with early exit (continue)")

=== ENHANCED EXTRACTION SUMMARY REPORT ===

📊 OVERALL STATISTICS
   Total JSON files processed: 2
   Total entries across all files: 60
   Total successful extractions: 48
   Overall success rate: 80.0%
   
🔍 QUALITY FILTERING RESULTS
   Successful sketches (raw): 48
   ✅ Qualified sketches (saved): 42
   ❌ Too short (< 50 lines): 3
   ❌ Missing mandatory variables: 3
   ❌ Missing output path: 0
   Quality pass rate: 87.5%

📁 FILES PROCESSED (Success → Qualified)
   raw_phi4_85a9_tpusg_batch.json:
     Success: 30/30 (100.0%)
     Qualified: 30/30 (100.0%)
   raw_phi4_85a9_psg_batch.json:
     Success: 18/30 (60.0%)
     Qualified: 12/18 (66.7%)
     ❌ Too short: 3
        - 7a05: 45 lines
        - e043: 3 lines
        - 5956: 3 lines
     ❌ Missing mandatory vars: 3
        - c802 (psg): models/ssd-mobilenet_v1/labelmap.txt
        - 8c80 (psg): data/object_detection/sheeps.mp4, results/object_detection/test_results/sheeps_detections.mp4
        - a0cf (psg): models/ssd-mobilenet_v1